# Modules & Data

In [1]:
import os
import pandas as pd 
from spellchecker import SpellChecker
from autocorrect import spell
import pkg_resources
from symspellpy.symspellpy import SymSpell, Verbosity  # import the module
import gensim.models
from collections import Counter
import math

## Ingredient

In [2]:
ingred = pd.read_csv(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\ingredients.csv')

## Recipe

In [4]:
recipe = pd.read_excel(r'./Recipes Masterfile_Dec 2019.xlsx')

# Note

$$\text{Word2Vec is the optimal choice}\$$

# Clean 

In [ ]:
ingreds = ingred.name.str.split(expand=True)

In [ ]:
ingreds

In [ ]:
for i in range(1,len(ingreds.columns)):
    ingreds_single = ingreds[0].append(ingreds[i], ignore_index = True)
    ingreds_single = ingreds_single.to_frame()

ingreds_single

In [ ]:
ingreds_single_count = ingreds_single.groupby(ingreds_single.loc[:,0]).size().sort_values(ascending=True)
ingreds_single_count.shape

In [ ]:
ingreds_single_count

In [ ]:
ingreds_single_clean = ingreds_single.drop_duplicates(keep='first', inplace=False).dropna()
ingreds_single_clean

In [ ]:
ingred_count
#webscrape not done ideally

In [ ]:
ingred.columns

## Truncate Data 

In [ ]:
ingreds = ingred.truncate(after = 5)
ingreds.shape

In [ ]:
ingreds_single_clean_truncate = ingreds_single_clean.truncate(after = 5)
ingreds_single_clean_truncate.shape

# Spell Checker 

In [ ]:
spell = SpellChecker()

# find those words that may be misspelled
misspell_list = []
correction_list = []
for i in ingreds.name:
    misspelled = spell.unknown(i)
    misspell_list.append(i)
    correction_list.append(spell.correction(i))
    # Get a list of `likely` options
    #print(spell.candidates(word))

In [ ]:
print(len(misspell_list))
misspell_list

In [ ]:
correction_list

# Word2Vec 

## Open the word2vec file 

In [6]:
import gensim

model3 = gensim.models.KeyedVectors.load_word2vec_format('../YoRipe/Dictionaries/GoogleNews-vectors-negative300.bin',
                                                        binary=True)


# Load Google's pre-trained Word2Vec model.
#model = gensim.models.Word2Vec('../YoRipe/Dictionaries/GoogleNews-vectors-negative300.bin')  
#model.save("word2vectrial.model")
#Word2Vec.load_word2vec_format
#KeyedVectors.load_word2vec_format

In [7]:
words = model3.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [8]:
WORDS

{'</s>': 0,
 'in': 1,
 'for': 2,
 'that': 3,
 'is': 4,
 'on': 5,
 '##': 6,
 'The': 7,
 'with': 8,
 'said': 9,
 'was': 10,
 'the': 11,
 'at': 12,
 'not': 13,
 'as': 14,
 'it': 15,
 'be': 16,
 'from': 17,
 'by': 18,
 'are': 19,
 'I': 20,
 'have': 21,
 'he': 22,
 'will': 23,
 'has': 24,
 '####': 25,
 'his': 26,
 'an': 27,
 'this': 28,
 'or': 29,
 'their': 30,
 'who': 31,
 'they': 32,
 'but': 33,
 '$': 34,
 'had': 35,
 'year': 36,
 'were': 37,
 'we': 38,
 'more': 39,
 '###': 40,
 'up': 41,
 'been': 42,
 'you': 43,
 'its': 44,
 'one': 45,
 'about': 46,
 'would': 47,
 'which': 48,
 'out': 49,
 'can': 50,
 'It': 51,
 'all': 52,
 'also': 53,
 'two': 54,
 'after': 55,
 'first': 56,
 'He': 57,
 'do': 58,
 'time': 59,
 'than': 60,
 'when': 61,
 'We': 62,
 'over': 63,
 'last': 64,
 'new': 65,
 'other': 66,
 'her': 67,
 'people': 68,
 'into': 69,
 'In': 70,
 'our': 71,
 'there': 72,
 'A': 73,
 'she': 74,
 'could': 75,
 'just': 76,
 'years': 77,
 'some': 78,
 'U.S.': 79,
 'three': 80,
 'million': 81

In [11]:
dictionary = []
for i in WORDS:
    dictionary.append([i])

len(dictionary)
dictionary

[['</s>'],
 ['in'],
 ['for'],
 ['that'],
 ['is'],
 ['on'],
 ['##'],
 ['The'],
 ['with'],
 ['said'],
 ['was'],
 ['the'],
 ['at'],
 ['not'],
 ['as'],
 ['it'],
 ['be'],
 ['from'],
 ['by'],
 ['are'],
 ['I'],
 ['have'],
 ['he'],
 ['will'],
 ['has'],
 ['####'],
 ['his'],
 ['an'],
 ['this'],
 ['or'],
 ['their'],
 ['who'],
 ['they'],
 ['but'],
 ['$'],
 ['had'],
 ['year'],
 ['were'],
 ['we'],
 ['more'],
 ['###'],
 ['up'],
 ['been'],
 ['you'],
 ['its'],
 ['one'],
 ['about'],
 ['would'],
 ['which'],
 ['out'],
 ['can'],
 ['It'],
 ['all'],
 ['also'],
 ['two'],
 ['after'],
 ['first'],
 ['He'],
 ['do'],
 ['time'],
 ['than'],
 ['when'],
 ['We'],
 ['over'],
 ['last'],
 ['new'],
 ['other'],
 ['her'],
 ['people'],
 ['into'],
 ['In'],
 ['our'],
 ['there'],
 ['A'],
 ['she'],
 ['could'],
 ['just'],
 ['years'],
 ['some'],
 ['U.S.'],
 ['three'],
 ['million'],
 ['them'],
 ['what'],
 ['But'],
 ['so'],
 ['no'],
 ['like'],
 ['if'],
 ['only'],
 ['percent'],
 ['get'],
 ['did'],
 ['him'],
 ['game'],
 ['back'],
 ['be

## Use the model 

In [6]:
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

## Word Test 

In [ ]:
correction_words= []
original_words = []
for i in ingreds_single_clean[0]:
    correction_words.append(correction(i))
    original_words.append(i)
    print(i, correction(i))

In [ ]:
counterA = Counter(original_words)
counterB = Counter(correction_words)


def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

print(counter_cosine_similarity(counterA, counterB) * 100)

## Phrases Test 

In [ ]:
correction_phrases= []
original_phrases = []
for i in recipe.ingredients:
    correction_phrases.append(correction(i))
    original_phrases.append(i)

In [ ]:
hello

In [8]:
counterA = Counter(original_phrases)
counterB = Counter(correction_phrases)


def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

print(counter_cosine_similarity(counterA, counterB) * 100)

99.51923076923075


In [10]:
recipe_tuples = list(zip(correction_phrases, original_phrases))
recipe_changed = pd.DataFrame(recipe_tuples, columns = ['Corrected','Original'])

In [13]:
list1 = list(set(correction_phrases) - set(original_phrases))
len(list1)

6

In [14]:
list2 = set(set(original_phrases) - set(correction_phrases))
len(list2)

6

In [15]:
len(list2)/len(original_phrases)

0.004862236628849271

In [16]:
recipe_changed['Changed'] = recipe_changed['Original'].apply(lambda  x: 1 if x in list2 else 0)

In [17]:
recipe_changed.loc[recipe_changed['Changed']==1]

,Corrected,Original,Changed
284,Chenggong,Cheng Tng,1
288,Chiappa,Chia Spa,1
417,Choonara,Choconana,1
814,Huckleberry,Huckle-Berry,1
885,Kimba,Kimbap,1
1231,Yucheng,Yu Sheng,1


# Export 

In [18]:
recipe_changed.to_excel(r'C:\Users\Solomonzhs\Desktop\Fresh\YoRipe\recipe_changed.xlsx')